### Objectives
This code file aims to generate many-to-many relations:
- Plant_with_package (many-to-many)
- ~~Plant_with_machine_type (many-to-many)~~
- Machine_type_with_operation_type (many-to-many)
- Chip_type_with_operation_type (This is generated in file ``operation_data_generation.ipynb``)
- Operation_with_machine (many-to-many)

In [1]:
import pandas as pd
import numpy as np
import random
import datetime

### Attributes of ``Plant_with_package``
Foreign key & Primary key: ``package_id``, ``plant_id``

In [51]:
package = pd.read_csv("dataset/package.csv")
all_pk_ids = list(package["package_id"])

plant = pd.read_csv("dataset/plant.csv")
all_plant_ids = list(plant["plant_id"])

package_id = []
plant_id = []
for pk_id in all_pk_ids:
    num_plants = random.randint(1, 5)
    selected_plant_ids = np.random.choice(all_plant_ids, num_plants, replace=False)
    for pl_id in selected_plant_ids:
        package_id.append(pk_id)
        plant_id.append(pl_id)
        
plant_with_package = pd.DataFrame(data={"package_id": package_id, "plant_id": plant_id})
plant_with_package.to_csv("dataset/plant_with_package.csv", index=False)

### ~~Attributes of ``Plant_with_machine_type``~~
~~Foreign key & Primary key: ``plant_id``, ``machine_name``, ``machine_version``~~

In [52]:
machine_type = pd.read_csv("dataset/machine_type.csv")
all_names = machine_type["machine_name"]
all_versions = machine_type["machine_version"]
all_ids = [i for i in range(len(all_names))]

# plant_id = []
# machine_name = []
# machine_version = []
# for pl_id in all_plant_ids:
#     num_machine_types = random.randint(1, 5)
#     selected_machine_id = np.random.choice(all_ids, num_machine_types, replace=False)
#     for id in selected_machine_id:
#         plant_id.append(pl_id)
#         machine_name.append(all_names[id])
#         machine_version.append(all_versions[id])
        
# plant_with_machine_type = pd.DataFrame(data={"plant_id": plant_id, "machine_name": machine_name, "machine_version": machine_version})
# plant_with_machine_type.to_csv("dataset/plant_with_machine_type.csv", index=False)

### Attributes of ``Machine_type_with_operation_type``
Primary key & Foreign key: ``operation_name``, ``machine_name``, ``machine_version``

Attributes: ``expense``, ``duration``

In [53]:
operation_type = pd.read_csv("dataset/operation_type.csv")
all_operation_names = operation_type["operation_name"]

operation_name = []
machine_name = []
machine_version = []
for id in all_ids:
    num_operation_types = random.randint(1, 5)
    selected_operation_names = np.random.choice(all_operation_names, num_operation_types, replace=False)
    for op_name in selected_operation_names:
        operation_name.append(op_name)
        machine_name.append(all_names[id])
        machine_version.append(all_versions[id])
    
machine_type_with_operation_type = pd.DataFrame(data={"operation_name": operation_name, "machine_name": machine_name, "machine_version": machine_version})
machine_type_with_operation_type.to_csv("dataset/machine_type_with_operation_type.csv", index=False)

### Attributes of ``operation_with_chip``
Primary key & Foreign key: ``operation_id``, ``chip_id``

In [54]:
chip_type_with_operation_type = pd.read_csv("dataset/chip_type_with_operation_type.csv")

chip_type = pd.read_csv("dataset/chip_type.csv")

chip = pd.read_csv("dataset/chip.csv")

operation = pd.read_csv("dataset/operation.csv")

operation_id = []
chip_id = []

operation_type_dict = dict()
operation_id_dict = dict()
chip_type_dict = dict()
for _, rows in chip_type.iterrows():
    key = rows["chip_name"] + str(rows["chip_version"])
    if key not in chip_type_dict:
        chip_type_dict[key] = []
    for _, rows2 in chip_type_with_operation_type.iterrows():
        if rows2["chip_name"] == rows["chip_name"] and rows2["chip_version"] == rows["chip_version"]:
            chip_type_dict[key].append(rows2["operation_name"])

for _, rows in operation.iterrows():
    if rows["operation_name"] not in operation_type_dict:
        operation_type_dict[rows["operation_name"]] = [rows["operation_id"]]
    else:
        operation_type_dict[rows["operation_name"]].append(rows["operation_id"])

for _, rows in chip.iterrows():
    key = rows["chip_name"] + str(rows["chip_version"])
    for op_name in chip_type_dict[key]:
        for id in operation_type_dict[op_name]:
            if id in operation_id_dict:
                continue
            chip_id.append(rows["chip_id"])
            operation_id.append(id)
            operation_id_dict[id] = 1
            break

operation_with_chip = pd.DataFrame(data={"operation_id": operation_id, "chip_id": chip_id})
operation_with_chip.to_csv("dataset/operation_with_chip.csv", index=False)

### Attributes of ``operation_with_machine``
Primary key & Foreign key: ``operation_id``, ``machine_id``

In [17]:
operation = pd.read_csv("dataset/operation.csv")

machine = pd.read_csv("dataset/machine.csv")
machine_dict = dict()
for _, rows in machine.iterrows():
    key = rows["machine_name"] + str(rows["machine_version"])
    if key not in machine_dict:
        machine_dict[key] = []
    machine_dict[key].append(rows["machine_id"])

operation_type_with_machine_type = pd.read_csv("dataset/machine_type_with_operation_type.csv")
op_name_dict = dict()
for _, rows in operation_type_with_machine_type.iterrows():
    if rows["operation_name"] not in op_name_dict:
        op_name_dict[rows["operation_name"]] = []
    op_name_dict[rows["operation_name"]].append(rows["machine_name"] + str(rows["machine_version"]))

operation_id = []
machine_id = []

machine_time = dict()

for _, rows in operation.iterrows():
    op_name = rows["operation_name"]
    for key in op_name_dict[op_name]:
        for id in machine_dict[key]:
            if id not in machine_time:
                machine_time[id] = datetime.datetime.strptime(rows["end_time"], "%Y-%m-%d %H:%M:%S")
                operation_id.append(rows["operation_id"])
                machine_id.append(id)
            else:
                if (datetime.datetime.strptime(rows["start_time"], "%Y-%m-%d %H:%M:%S") - machine_time[id]).seconds > 0:
                    machine_time[id] = datetime.datetime.strptime(rows["end_time"], "%Y-%m-%d %H:%M:%S")
                    operation_id.append(rows["operation_id"])
                    machine_id.append(id)

operation_with_machine = pd.DataFrame(data={"operation_id": operation_id, "machine_id": machine_id})
operation_with_machine.to_csv("dataset/operation_with_machine.csv", index=False)